In [ ]:
import heapq
import random
import time
from heapq import heappush, heappop

# --- Priority Queue for Dynamic Slot Allocation ---
class Flight:
    def __init__(self, id, urgency, fuel_level, emergency=False):
        self.id = id
        self.urgency = urgency
        self.fuel_level = fuel_level
        self.emergency = emergency
        self.priority_score = self.calculate_priority()

    def calculate_priority(self):
        # Priority score: urgency + fuel_level (emergency status makes it higher priority)
        return self.urgency + (self.fuel_level * 0.5) + (100 if self.emergency else 0)

    def __lt__(self, other):
        # This will ensure that the flight with higher priority score comes first
        return self.priority_score > other.priority_score

# --- Airspace Graph Management (Using Dijkstra's Algorithm) ---
class Airspace:
    def __init__(self):
        self.graph = {}
    
    def add_edge(self, from_node, to_node, cost):
        if from_node not in self.graph:
            self.graph[from_node] = []
        self.graph[from_node].append((to_node, cost))
    
    def dijkstra(self, start, end):
        # Dijkstra's shortest path algorithm
        shortest_paths = {start: (None, 0)}  # node: (previous_node, cost)
        current_node = start
        visited = set()
        
        while current_node != end:
            visited.add(current_node)
            destinations = self.graph.get(current_node, [])
            weight_to_current_node = shortest_paths[current_node][1]
            
            for next_node, weight in destinations:
                if next_node in visited:
                    continue
                old_cost = shortest_paths.get(next_node, (None, float('inf')))[1]
                new_cost = weight_to_current_node + weight
                
                if new_cost < old_cost:
                    shortest_paths[next_node] = (current_node, new_cost)
                
            next_destinations = {node: cost for node, (previous_node, cost) in shortest_paths.items()}
            if not next_destinations:
                return None
            current_node = min(next_destinations, key=next_destinations.get)
        
        path = []
        while current_node is not None:
            path.append(current_node)
            next_node = shortest_paths[current_node][0]
            current_node = next_node
        path = path[::-1]
        return path

# --- Runway Scheduling Algorithm ---
class Runway:
    def __init__(self):
        self.flights_queue = []
        heapq.heapify(self.flights_queue)  # Min-heap

    def schedule_flight(self, flight):
        heappush(self.flights_queue, flight)

    def takeoff_or_land(self):
        if self.flights_queue:
            flight = heappop(self.flights_queue)
            print(f"Flight {flight.id} scheduled for {'landing' if flight.urgency < 5 else 'takeoff'}.")

# --- Simulate Flight Operations ---
def simulate_air_traffic():
    airspace = Airspace()
    runway = Runway()
    
    # Example airspace nodes and flight paths
    airspace.add_edge('Approach1', 'Runway1', 10)
    airspace.add_edge('Approach2', 'Runway1', 15)
    airspace.add_edge('Approach1', 'Runway2', 5)
    airspace.add_edge('Approach2', 'Runway2', 10)
    
    # Flights waiting to take off or land
    flights = [Flight(f"Flight_{i}", random.randint(1, 10), random.randint(30, 100), random.choice([True, False])) for i in range(10)]
    
    print("Flights before scheduling:")
    for flight in flights:
        print(f"Flight {flight.id}, Priority Score: {flight.priority_score}")
    
    # Schedule flights based on priority
    for flight in flights:
        runway.schedule_flight(flight)
    
    print("\nFlight scheduling based on priority:")
    while runway.flights_queue:
        runway.takeoff_or_land()
        time.sleep(1)

    # Airspace management (find optimal route for a flight)
    flight_path = airspace.dijkstra('Approach1', 'Runway1')
    print("\nOptimal path for a flight:", flight_path)

# Run the simulation
simulate_air_traffic()


Flights before scheduling:
Flight Flight_0, Priority Score: 19.0
Flight Flight_1, Priority Score: 128.5
Flight Flight_2, Priority Score: 34.5
Flight Flight_3, Priority Score: 128.5
Flight Flight_4, Priority Score: 56.5
Flight Flight_5, Priority Score: 43.0
Flight Flight_6, Priority Score: 139.5
Flight Flight_7, Priority Score: 150.0
Flight Flight_8, Priority Score: 145.5
Flight Flight_9, Priority Score: 45.0

Flight scheduling based on priority:
Flight Flight_7 scheduled for landing.
Flight Flight_8 scheduled for takeoff.
Flight Flight_6 scheduled for takeoff.
Flight Flight_1 scheduled for takeoff.
Flight Flight_3 scheduled for takeoff.
Flight Flight_4 scheduled for takeoff.
Flight Flight_9 scheduled for takeoff.
Flight Flight_5 scheduled for takeoff.
Flight Flight_2 scheduled for landing.
Flight Flight_0 scheduled for landing.
